In [2]:
from PyPDF2 import PdfReader
import pymupdf
import numpy as np
import cv2
import shutil
import imageio
from PIL import Image
import imagehash
import tempfile
import os
from llama_index.core.indices import MultiModalVectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import SimpleDirectoryReader, StorageContext
# from awsfunctions import upload_folder_to_s3, check_file_exists_in_s3, download_folder_from_s3, delete_s3_folder
import qdrant_client
import streamlit as st

In [3]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    full_text = ''
    for page in reader.pages:
        text = page.extract_text()
        full_text += text
    return full_text

In [4]:
full_text = extract_text_from_pdf('/Users/pushpanjali/samsung/sample.pdf')

In [8]:
import pdfplumber

def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Usage example
pdf_file_path = "/Users/pushpanjali/samsung/sample.pdf"
extracted_text = extract_text_from_pdf(pdf_file_path)

if extracted_text:
    print("Extracted text:")
    print(extracted_text[:100] + "...")  # Print first 100 characters
else:
    print("Failed to extract text.")


Failed to extract text.


In [7]:
! pip install pdfplumber

  Using cached pdfplumber-0.11.4-py3-none-any.whl.metadata (41 kB)
  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.0-py3-none-macosx_11_0_arm64.whl.metadata (48 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached pdfplumber-0.11.4-py3-none-any.whl (59 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
Using cached pypdfium2-4.30.0-py3-none-macosx_11_0_arm64.whl (2.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 1.9 MB/s eta 0:00:00a 0:00:01
Using cached pycparser-2.22-py3-none-any.whl (117 kB)


In [9]:
text = """1. 일반사항
1.1 적용범위
(1) 이 기준은 법령 및 규정의 준수, 수급인의 기본 의무, 현장 확인 및 설계도서 검토,
책임한계, 착수 전 합동조사, 시공 전 협의, 공사수행, 야간공사, 동절기 공사, 하도급
관리, 공사협의 및 조정 등에 대해서 적용한다.
1.2 참고 기준
1.2.1 관련법규
건설기술진흥법령
건설산업기본법령
공사계약일반조건
야간 건설공사 안전보건작업 지침 (한국산업안전보건공단)
1.2.2 관련기준
내용 없음.
1.3 용어의 정의
∙발주자: 건설산업기본법 제2조제10호의 발주자를 말한다.
∙공사감독자: 공사계약일반조건 제2조제3호의 공사감독관을 말한다.
∙수급인: 공사계약 일반조건 제2조제2호의 계약상대자를 말한다.
∙하수급인: 건설산업기본법 제2조제14호의 하수급인을 말한다.
∙현장대리인: 공사계약일반조건 제14조에 따라 공사현장대리인으로서, 현장에서 전반적
인 공사 업무와 관리를 책임 있게 시행할 수 있는 권한을 가진 건설 기술자를 말한다.
∙설계서: 공사계약일반조건 제2조제4호의 설계서를 말한다.
∙검사: 공사계약문서에 기재된 시공 단계 또는 납품된 공사재료 등의 품질 확보를 위해
기성부분 또는 완성품의 품질, 규격, 수량 등을 확인하는 것을 말한다.
∙승인: 수급인이 제출, 신고 등의 방법으로 요청한 사항에 대해 공사감독자가 권한범위
내에서 서면으로 동의하는 것을 말한다.
∙지시: 공사감독자가 권한 범위 내에서 필요한 사항을 수급인에게 실시하도록 지시하는
것을 말한다.
∙확인: 계약문서대로 공사를 실시하고 있는지, 또는 지시, 조정, 승인, 검사 이후에 원래
의도와 규정대로 시행되었는지를 공사감독자가 확인하는 것을 말한다.


제목: 일반사항
1.4 해석
(1) 표준시방서(또는 공사시방서)에서 사용된 용어의 해석은 아래 순서에 따른다.
(2) 계약문서 (표준시방서(또는 공사시방서) 포함)
(3) 건설기술진흥법과 동 시행령, 동 시행규칙
(4) 기타 건설관련 법규
(5) 공사 종류별 용어사전
(6) 국어사전
1.5 적용순서
(1) 공사시방서에서 KCS 10 10 05, KCS 10 10 10, KCS 10 10 15, KCS 10 10 20, KCS
10 10 25, KCS 10 10 30, KCS 10 10 35와 이 기준들 외의 시방기준 내용간에 상호모
순이 있을 경우에는 이 기준들 외의 시방기준에 명시된 내용을 우선 적용한다.
1.6 법령 및 규칙의 준수
(1) 수급인은 공사와 관련된 모든 법률, 시행령, 시행규칙, 훈령 및 예규, 조례 및 규칙 등
(이하 건설관련법령)을 준수하여야 한다.
(2) 수급인은 표준시방서(또는 공사시방서)의 내용이 건설관련법령과 상호 모순될 경우(건
설공사 중에 건설관련법령이 변경되고 변경된 규정에 따라야 할 경우를 포함한다.)에
는 건설관련법령을 우선하여 준수하여야 한다.
(3) 수급인은 자신이나 고용인이 건설관련법령과 계약문서에 규정된 내용을 준수하여 시
공하도록 해야 하며 이의 위반으로 민원이 발생하지 않도록 해야 한다.
1.7 수급인의 기본 의무
(1) 수급인은 계약에 따라 공사를 성실하게 이행해야 하며, 하자가 발생할 경우 건설관련
법령 및 계약문서에 따라 성실하게 보수하여야 한다.
(2) 수급인은 특별한 사유가 없는 한 공사감독자의 업무수행에 적극적인 자세로 협조하여
야 한다.


제목: 일반사항
1.8 현장 확인 및 설계도서 검토
(1) 수급인은 공사 착수 전에 건설기술진흥법 제48조제2항에 따라 설계도서를 면밀히 검
토하고, 설계도서의 오류, 누락 등으로 공사가 잘못되거나 공기가 지연되는 일이 없도
록 조치하여야 한다.
(2) 설계도서를 검토하고 아래와 같은 경우가 있으면 수급인의 현장대리인은 검토의견서를 첨
부하여 발주자에 통지하고 발주자의 해석 또는 지시를 받은 후 공사를 시행하여야 한다.
① 설계도서의 내용이 현장 조건과 일치하는지 여부
② 설계도서대로 시공할 수 있는지 여부
③ 그 밖에 시공과 관련된 사항
④ 하자발생이 우려되는 경우
⑤ 설계변경 사유 및 계약기간연장 사유가 있는 경우
⑥ 품질향상이나 공사비 절감을 기할 수 있는 경우
(3) 수급인이 발주자에게 통지하지 않았거나 발주자의 해석 또는 지시를 받기 전에 임의
로 수행한 공사는 기성량으로 인정하지 않는다. 또한 수급인이 임의로 시행한 공사에
대해 공사감독자의 원상복구나 시정 지시가 있는 경우 수급인은 수급인 부담으로 즉
시 이행하여야 한다.
1.9 책임한계
(1) 수급인은 계약문서를 준수하여 공사를 이행해야 하며, 발주자의 시정요구 또는 이행
촉구지시가 있을 때에는 이에 따라야 한다. 또한, 수급인은 인허가 변경, 민원 및 협
의결과 등으로 인해 설계서를 변경할 필요가 있어 발주자가 설계변경을 요청할 경우
에는 이에 성실히 응하여야 한다.
(2) 수급인은 건설공사와 관련하여 정부, 발주자, 외부기관 등에서 시행하는 각종 평가,
감사, 점검의 수감과 이에 따른 시정 지시를 성실히 이행해야 한다.
(3) 수급인은 현장대리인 등 수급인이 해당 공사를 위하여 임명, 지정, 고용한 자 및 수급
인과 납품계약 또는 하도급 계약 체결한 자의 공사 관련 행위 및 결과에 대한 일체의
책임을 져야 한다.
(4) 수급인은 공사 현장의 이용 및 작업 효율 증대, 품질 향상, 안전사고와 환경공해 예
방, 보건위생 등을 위하여 현장과 주변을 청결하게 유지하여야 한다.
(5) 수급인이 발주자와 공사감독자에게 하는 보고, 통지, 요청, 문제 또는 이의 제기 등은
서면으로 해야 효력이 있다.
(6) 인⋅허가 사항은 발주자가 수행함 을 원칙으로 하며, 수급인은 원활한 업무수행을 위
하여 인⋅허가 업무에 최대한의 협조와 지원을 하여야 한다.


제목: 일반사항
1.10 착수 전 합동조사
(1) 수급인은 구조물, 부대시설 등 해당 공종의 공사착수 전에 관계기관(행정 및 유관 기
관), 지역 주민대표, 공사감독자와 합동으로 설계도서상 내용과 현장의 적합 여부를
조사하여야 한다.
(2) 수급인은 조사결과에 따라 변경될 사항에 대하여 사유, 변경방안, 변경내용 등을 작성
하여 발주자 또는 공사감독자에게 보고하여야 한다. 발주자 또는 공사감독자는 조사
결과에 따라 구조물 및 부대시설의 위치, 규격 등을 종합적으로 검토하여 변경과 추
가 설치의 필요성이 인정될 경우 설계변경에 반영할 수 있도록 조치하여야 한다.
1.11 시공 전 협의
1.11.1 공사 합동회의
공사감독자는 공사 착수일로부터 1개월 이내에 최초 공사관련자 합동회의를 개최해야 하
며, 이 회의에서 각각의 책임한계를 검토하고, 필요한 회의별로 장소, 일시, 참석범위, 월
개최 횟수 등을 정한다.
1.11.2 공사추진 합동회의
공사감독자는 각 공사의 특수사항 및 사전 협의사항 등 업무를 조정하기 위하여 공사추
진 합동회의를 개최하여야 한다.
1.12 공사수행
1.12.1 공사수행 일반
(1) 수급인은 계약문서에 명시되지 않은 사항이라도 구조상 또는 외관상 당연히 시공을
요하는 부분은 발주자 및 공사감독자와 협의하여 이행하여야 한다.
(2) 발주자는 관련 법령, 계약문서에 의하여 자재 등의 품질 및 시공이 적정하지 못하다
고 인정되는 경우에는 재시공 등의 지시를 할 수 있으며, 수급인은 이에 따라야 한다.
1.12.2 공사감독자의 업무
공사계약일반조건 제16조(공사감독관)에 따른다.
1.12.3 응급조치
공사계약일반조건 제24조(응급조치)에 따른다.
1.12.4 지중 발굴물
공사계약일반조건 제38조(발굴물의 처리)에 따른다.


제목: 일반사항
1.13 야간공사
(1) 야간공사는 안전사고, 품질확보 불리 등의 문제로 시행하지 않음이 원칙이나 민원발
생, 교통대책 등으로 불가피하게 시행할 경우에는 다음 사항에 유의하여 품질확보, 부
실공사 방지, 안전관리에 만전을 기해야 한다. 또한 발주자의 지시가 있는 경우 발주
자와 협의하여 추가비용을 청구할 수 있다.
(2) 건설기술진흥법 시행령 제98조제1항에 규정된 안전관리계획 수립대상인 건설공사에서
야간공사를 시행하여야 할 경우 야간공사가 고려된 안전관리계획을 수립해야 한다.
(3) 공사장 조명, 작업자 복장, 안전표시 방법 및 기준, 야간공사 안전시설 기준, 야간공사
작업자 건강관리 및 야간공사 안전조치 등의 세부사항은 안전보건공단 안전보건기술
지침인 야간 건설공사 안전보건작업지침(C-52-2012)에 따른다.
1.14 동절기 공사
(1) 동절기 공사 중단 기간에는 물을 사용하는 공사와 기온저하로 인하여 시공품질 확보
가 어려운 공사는 중단해야 한다. 다만, 다음의 경우에는 그러하지 아니한다.
(2) 수급인이 부득이한 사유가 있어 공사를 계속하여야 할 경우에는 동절기 공사로 인하
여 시공품질의 저하 및 안전사고 등을 충분히 예방할 수 있도록 동절기공사 시행방안
을 수립하여 발주자의 승인을 받은 후에 공사를 계속하여야 한다. 이 때 수급인은 추
가되는 비용을 발주자에게 청구할 수 없으며, 이 기간 동안의 공사시행이 원인이 되
어 발생하는 공사물의 잘못, 재시공 및 하자보수에 대한 책임을 져야 한다.
(3) 발주자로부터 공사를 계속하라는 지시가 있는 경우에 수급인은 지체 없이 동절기 공
사 시행방안을 수립하여 발주자의 승인을 받은 후에 공사를 계속하여야 한다. 수급인
은 이 기간 동안의 공사 시행이 원인이 되어 공사 결과물에 문제가 발생하거나, 재
시공의 필요 또는 하자 등이 발생할 경우에는 이에 대한 보수의 책임을 져야 한다.
또한 동절기 공사의 추가비용은 발주자와 협의하여 청구할 수 있다.
1.15 하도급 관리
(1) 수급인은 계약된 공사의 일부를 제3자에게 하도급 하고자 할 때에는 발주자의 서면승
인을 받아야 한다. 다만, 전문공사를 해당 전문공사업자에게 하도급하는 경우에는 건
설산업기본법 제29조제4항에 의하여 발주자에게 통지해야 한다.
(2) (1)의 규정에 의하여 하도급한 경우에도 계약상의 수급인 책임과 의무가 면제되지 않
으며, 수급인은 하수급인, 하수급인의 대리인, 하수급인이 채용한 근로자의 행위에 대
하여 모든 책임을 져야 한다.
(3) 수급인이 공사 일부를 하도급 하는 경우에는 공사 시행에 적합한 기술과 능력을 가진
자를 하수급인으로 선정하여야 한다.
(4) 수급인은 하도급을 시행하기 전에 건설산업기본법 제31조의2에 따라 하도급계획서를
발주자에 제출하여야 한다.


제목: 일반사항
1.16 공사협의 및 조정
1.16.1 협의 및 조정
수급인은 해당 공사와 관련된 다른 공사 수급인들과 마찰을 방지하고 전체 공사가 계획
대로 완성될 수 있게 협력하고 최선의 방안을 도출한 후에 공사를 시행해야 한다. 이를
위해 관련 공사와의 접속부위 적합성, 공사 시공한계, 시공순서, 공사 착수시기, 공사 진
행속도, 공사 준비, 공사물 보호, 가시설물 등의 적합성에 대해 모든 공사 관련자들과 면
밀히 협의하고 조정하여 공사전체의 진행에 지장이 없도록 해야 한다.
1.16.2 발주자의 조속 완공 또는 연기 요구에 대한 조치
발주자는 공사의 안전, 일반인 보호, 2인 이상의 수급인이 관련된 공사의 원활한 수행을
위하여 당해 건설공사 일부분의 조속한 완공 또는 연기를 요구할 수 있으며, 수급인은 특
별한 사유가 없는 한 이에 따라야 한다. 발주자는 이에 소요되는 추가비용을 수급인에게
지급할 수 있다.
1.16.3 협의 및 조정에 따른 설계변경
(1) 수급인은 해당 공사와 연관된 다른 공사와의 상호 마찰방지를 위한 협의⋅조정 결과
가 아래와 같은 경우 발주자에 설계변경을 요청할 수 있다.
① 지하구조물 공사의 우선순위 상 불가피한 선⋅후 시공에 따른 기초저면의 안전성
저하를 방지하기 위해 설계변경이 불가피한 경우
② 광통신관로, 공동구, 전화 및 전선관로, 배수관, 급수관 등 지하매설물의 교차, 존
재 유무 등에 의해 매설심도가 변경되어 설계변경이 불가피한 경우
1.16.4 협의 및 조정에 대한 수급인의 책임
수급인은 공사 상호간의 협의 및 조정을 소홀히 하여 발생한 재시공 또는 수정⋅보완 공
사에 대해 책임을 져야 한다.
1.16.5 종합 공정관리에 협조
수급인은 착공부터 준공까지 토목, 건축, 기계, 전기, 통신, 조경, 급배수, 도시가스, 전기
통신관로 공사 등은 물론 타 행정기관 등과의 협조, 관련 공사 전체의 원활한 추진을 위
해 공사감독자가 요구하는 종합 공정관리 계획 및 운영에 적극 협조해야 한다."""

In [11]:
type(text)

str

In [15]:
def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding("o200k_base") # Its for [gpt-4, gpt-3.5-turbo, text-embedding-ada-002] models
    num_tokens = len(encoding.encode(string))
    return num_tokens


In [16]:
import tiktoken
num_tokens_from_string(text)

3803